In [3]:
import cv2
import numpy as np

# Video path
video_path = "F:\Shady\FCAI\Fouth year\image processing\Project\Project Video.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get video properties
width = int(cap.get(3))
height = int(cap.get(4))
fps = cap.get(5)

# Create a window to display the processed video with bounding boxes
cv2.namedWindow("Bounding Boxes", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Bounding Boxes", width, height)

while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    if not ret:
        break

    # Preprocess the frame (e.g., resizing, blurring, thresholding)
    processed_frame = cv2.resize(frame, (width, height))  # Resize to the original size
    processed_frame = cv2.GaussianBlur(processed_frame, (5, 5), 0)  # Apply Gaussian Blur
    _, processed_frame = cv2.threshold(processed_frame, 127, 255, cv2.THRESH_BINARY)  # Thresholding

    # Assume subtitles are typically at the bottom of the screen
    subtitle_region = processed_frame[int(0.9 * height):, :]

    # Convert the subtitle region to grayscale for contour detection
    subtitle_gray = cv2.cvtColor(subtitle_region, cv2.COLOR_BGR2GRAY)

    # Create a mask for the subtitles (assuming white subtitles on a dark background)
    subtitle_mask = cv2.inRange(subtitle_gray, 200, 255)

    # Set the subtitle region pixels outside the mask to black
    subtitle_region[np.where(subtitle_mask == 0)] = 0

    # Find contours in the subtitle region
    contours, _ = cv2.findContours(subtitle_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through contours and draw bounding boxes for subtitles
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Draw a bounding box for the subtitle in red
        cv2.rectangle(frame, (x, int(0.9 * height) + y), (x + w, int(0.9 * height) + y + h), (0, 0, 255), 2)

        # Extract the region of interest (ROI) for the subtitle
        subtitle_roi = processed_frame[int(0.9 * height) + y:int(0.9 * height) + y + h, x:x + w]

        # Convert the subtitle ROI to grayscale for contour detection
        subtitle_roi_gray = cv2.cvtColor(subtitle_roi, cv2.COLOR_BGR2GRAY)

        # Draw bounding boxes for words (example)
        words_contours, _ = cv2.findContours(subtitle_roi_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for word_contour in words_contours:
            word_x, word_y, word_w, word_h = cv2.boundingRect(word_contour)
            # Draw a bounding box for the word in green
            cv2.rectangle(frame, (x + word_x, int(0.9 * height) + y + word_y),
                          (x + word_x + word_w, int(0.9 * height) + y + word_y + word_h), (0, 255, 0), 1)

            # Further process the word bounding box to identify characters (if needed)
            # (You can use a similar approach as above with contour detection)

    # Display the processed frame with bounding boxes
    cv2.imshow("Bounding Boxes", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()
